## Library

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import KMeans
from sklearn.datasets import load_iris
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from sklearn.metrics import silhouette_score

##Masukkan Data

In [ ]:
url = "https://raw.githubusercontent.com/jhnwlkn/Machine-Leaning/main/customer_support_data.csv"
df = pd.read_csv(url)

<ipython-input-28-7ed4bae57010>:2: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(url)


In [ ]:
dtype_dict = {
    'Unique id': str,
    'channel_name': str,
    'category': str,
    'Sub-category': str,
    'Customer Remarks': str,
    'Order_id': str,
    'order_date_time': str,
    'Issue_reported at': str,
    'issue_responded': str,
    'Survey_response_Date': str,
    'Customer_City': str,
    'Product_category': str,
    'Item_price': float,
    'connected_handling_time': float,
    'Agent_name': str,
    'Supervisor': str,
    'Manager': str,
    'Tenure Bucket': str,
    'Agent Shift': str,
    'CSAT Score': float
}

In [ ]:
data = pd.read_csv(url, dtype=dtype_dict)

In [ ]:
df.drop(index=range(10527, 85906+1), inplace=True)

##EDA (Exploratory Dataa Analysis)

In [ ]:
df.head()

/usr/local/lib/python3.10/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,Customer_City,Product_category,Item_price,connected_handling_time,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score
0,7e9ae164-6a8b-4521-a2d4-58f7c9fff13f,Outcall,Product Queries,Life Insurance,NaN,c27c9bb4-fa36-4140-9f1f-21009254ffdb,NaN,1/8/2023 11:13,1/8/2023 11:47,1-Aug-23,NaN,NaN,NaN,NaN,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,Morning,5.0
1,b07ec1b0-f376-43b6-86df-ec03da3b2e16,Outcall,Product Queries,Product Specific Information,NaN,d406b0c7-ce17-4654-b9de-f08d421254bd,NaN,1/8/2023 12:52,1/8/2023 12:54,1-Aug-23,NaN,NaN,NaN,NaN,Vicki Collins,Dylan Kim,Michael Lee,>90,Morning,5.0
2,200814dd-27c7-4149-ba2b-bd3af3092880,Inbound,Order Related,Installation/demo,NaN,c273368d-b961-44cb-beaf-62d6fd6c00d5,NaN,1/8/2023 20:16,1/8/2023 20:38,1-Aug-23,NaN,NaN,NaN,NaN,Duane Norman,Jackson Park,William Kim,On Job Training,Evening,5.0
3,eb0d3e53-c1ca-42d3-8486-e42c8d622135,Inbound,Returns,Reverse Pickup Enquiry,NaN,5aed0059-55a4-4ec6-bb54-97942092020a,NaN,1/8/2023 20:56,1/8/2023 21:16,1-Aug-23,NaN,NaN,NaN,NaN,Patrick Flores,Olivia Wang,John Smith,>90,Evening,5.0
4,ba903143-1e54-406c-b969-46c52f92e5df,Inbound,Cancellation,Not Needed,NaN,e8bed5a9-6933-4aff-9dc6-ccefd7dcde59,NaN,1/8/2023 10:30,1/8/2023 10:32,1-Aug-23,NaN,NaN,NaN,NaN,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,Morning,5.0


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10527 entries, 0 to 10526
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unique id                10527 non-null  object 
 1   channel_name             10527 non-null  object 
 2   category                 10527 non-null  object 
 3   Sub-category             10527 non-null  object 
 4   Customer Remarks         3423 non-null   object 
 5   Order_id                 6030 non-null   object 
 6   order_date_time          1698 non-null   object 
 7   Issue_reported at        10527 non-null  object 
 8   issue_responded          10527 non-null  object 
 9   Survey_response_Date     10527 non-null  object 
 10  Customer_City            1684 non-null   object 
 11  Product_category         1696 non-null   object 
 12  Item_price               1698 non-null   float64
 13  connected_handling_time  10 non-null     float64
 14  Agent_name            

In [ ]:
df.isna().sum()

Unique id                      0
channel_name                   0
category                       0
Sub-category                   0
Customer Remarks            7104
Order_id                    4497
order_date_time             8829
Issue_reported at              0
issue_responded                0
Survey_response_Date           0
Customer_City               8843
Product_category            8831
Item_price                  8829
connected_handling_time    10517
Agent_name                     0
Supervisor                     0
Manager                        0
Tenure Bucket                  0
Agent Shift                    0
CSAT Score                     0
dtype: int64

In [ ]:
df.describe()

,Item_price,connected_handling_time,CSAT Score
count,1698.000000,10.000000,10527.000000
mean,6516.472909,298.100000,4.207372
std,13201.741722,173.490602,1.416707
min,9.000000,0.000000,1.000000
25%,426.500000,205.250000,4.000000
50%,999.000000,271.500000,5.000000
75%,4419.000000,403.250000,5.000000
max,134999.000000,574.000000,5.000000


In [ ]:
df.head(5)

/usr/local/lib/python3.10/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,Customer_City,Product_category,Item_price,connected_handling_time,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score
0,7e9ae164-6a8b-4521-a2d4-58f7c9fff13f,Outcall,Product Queries,Life Insurance,NaN,c27c9bb4-fa36-4140-9f1f-21009254ffdb,NaN,1/8/2023 11:13,1/8/2023 11:47,1-Aug-23,NaN,NaN,NaN,NaN,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,Morning,5.0
1,b07ec1b0-f376-43b6-86df-ec03da3b2e16,Outcall,Product Queries,Product Specific Information,NaN,d406b0c7-ce17-4654-b9de-f08d421254bd,NaN,1/8/2023 12:52,1/8/2023 12:54,1-Aug-23,NaN,NaN,NaN,NaN,Vicki Collins,Dylan Kim,Michael Lee,>90,Morning,5.0
2,200814dd-27c7-4149-ba2b-bd3af3092880,Inbound,Order Related,Installation/demo,NaN,c273368d-b961-44cb-beaf-62d6fd6c00d5,NaN,1/8/2023 20:16,1/8/2023 20:38,1-Aug-23,NaN,NaN,NaN,NaN,Duane Norman,Jackson Park,William Kim,On Job Training,Evening,5.0
3,eb0d3e53-c1ca-42d3-8486-e42c8d622135,Inbound,Returns,Reverse Pickup Enquiry,NaN,5aed0059-55a4-4ec6-bb54-97942092020a,NaN,1/8/2023 20:56,1/8/2023 21:16,1-Aug-23,NaN,NaN,NaN,NaN,Patrick Flores,Olivia Wang,John Smith,>90,Evening,5.0
4,ba903143-1e54-406c-b969-46c52f92e5df,Inbound,Cancellation,Not Needed,NaN,e8bed5a9-6933-4aff-9dc6-ccefd7dcde59,NaN,1/8/2023 10:30,1/8/2023 10:32,1-Aug-23,NaN,NaN,NaN,NaN,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,Morning,5.0


In [ ]:
data = data.fillna(0)

##Preprocessing

In [ ]:
df = df.drop(['Unique id','Customer Remarks','Order_id','order_date_time','Customer_City','Product_category','Item_price','connected_handling_time','Agent_name'],axis=1)

In [ ]:
df

In [ ]:
df['channel_name'] =  df['channel_name'].apply(lambda x: df['channel_name'].unique().tolist().index(x))
df['category'] =  df['category'].apply(lambda x: df['category'].unique().tolist().index(x))
df['Sub-category'] =  df['Sub-category'].apply(lambda x: df['Sub-category'].unique().tolist().index(x))
df['Issue_reported at'] =  df['Issue_reported at'].apply(lambda x: df['Issue_reported at'].unique().tolist().index(x))
df['issue_responded'] =  df['issue_responded'].apply(lambda x: df['issue_responded'].unique().tolist().index(x))
df['Survey_response_Date'] =  df['Survey_response_Date'].apply(lambda x: df['Survey_response_Date'].unique().tolist().index(x))
df['Supervisor'] =  df['Supervisor'].apply(lambda x: df['Supervisor'].unique().tolist().index(x))
df['Manager'] =  df['Manager'].apply(lambda x: df['Manager'].unique().tolist().index(x))
df['Tenure Bucket'] =  df['Tenure Bucket'].apply(lambda x: df['Tenure Bucket'].unique().tolist().index(x))
df['Agent Shift'] =  df['Agent Shift'].apply(lambda x: df['Agent Shift'].unique().tolist().index(x))
df['CSAT Score'] =  df['CSAT Score'].apply(lambda x: df['CSAT Score'].unique().tolist().index(x))

In [ ]:
df

/usr/local/lib/python3.10/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,Customer_City,Product_category,Item_price,connected_handling_time,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score
0,7e9ae164-6a8b-4521-a2d4-58f7c9fff13f,Outcall,Product Queries,Life Insurance,NaN,c27c9bb4-fa36-4140-9f1f-21009254ffdb,NaN,1/8/2023 11:13,1/8/2023 11:47,1-Aug-23,NaN,NaN,NaN,NaN,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,Morning,5.0
1,b07ec1b0-f376-43b6-86df-ec03da3b2e16,Outcall,Product Queries,Product Specific Information,NaN,d406b0c7-ce17-4654-b9de-f08d421254bd,NaN,1/8/2023 12:52,1/8/2023 12:54,1-Aug-23,NaN,NaN,NaN,NaN,Vicki Collins,Dylan Kim,Michael Lee,>90,Morning,5.0
2,200814dd-27c7-4149-ba2b-bd3af3092880,Inbound,Order Related,Installation/demo,NaN,c273368d-b961-44cb-beaf-62d6fd6c00d5,NaN,1/8/2023 20:16,1/8/2023 20:38,1-Aug-23,NaN,NaN,NaN,NaN,Duane Norman,Jackson Park,William Kim,On Job Training,Evening,5.0
3,eb0d3e53-c1ca-42d3-8486-e42c8d622135,Inbound,Returns,Reverse Pickup Enquiry,NaN,5aed0059-55a4-4ec6-bb54-97942092020a,NaN,1/8/2023 20:56,1/8/2023 21:16,1-Aug-23,NaN,NaN,NaN,NaN,Patrick Flores,Olivia Wang,John Smith,>90,Evening,5.0
4,ba903143-1e54-406c-b969-46c52f92e5df,Inbound,Cancellation,Not Needed,NaN,e8bed5a9-6933-4aff-9dc6-ccefd7dcde59,NaN,1/8/2023 10:30,1/8/2023 10:32,1-Aug-23,NaN,NaN,NaN,NaN,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,Morning,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10522,3e5424f6-1134-4228-8a0a-dfcce16e7477,Inbound,Returns,Reverse Pickup Enquiry,Good,9e97d6a3-4c68-4654-903f-50cba17cf7e6,NaN,5/8/2023 22:57,5/8/2023 23:00,5-Aug-23,NaN,NaN,NaN,NaN,Carolyn Reese,Madison Kim,Michael Lee,31-60,Evening,5.0
10523,a006368a-46f2-4260-84fd-b48a7e6f8730,Inbound,Product Queries,Product Specific Information,Very nice,d07fdfdb-bae7-492a-804b-fa546f37ea1a,NaN,6/8/2023 15:59,6/8/2023 16:11,6-Aug-23,NaN,NaN,NaN,NaN,Colleen Savage,Ava Wong,William Kim,On Job Training,Evening,5.0
10524,26f2d90b-4e70-4228-a30c-4d23d2597b8e,Inbound,Returns,Reverse Pickup Enquiry,NaN,808bdcc8-e64d-4808-9dc4-7024726dac9b,NaN,5/8/2023 7:38,5/8/2023 7:38,5-Aug-23,NaN,NaN,NaN,NaN,Rachel Peters,Aiden Patel,John Smith,>90,Morning,5.0
10525,4388f9b2-58fa-43f9-bf1d-ef91c85b4261,Inbound,Product Queries,Product Specific Information,NaN,9d6addee-d279-45da-8cc9-bc258f540048,NaN,5/8/2023 13:53,5/8/2023 14:00,5-Aug-23,NaN,NaN,NaN,NaN,Mark Garcia,Wyatt Kim,Jennifer Nguyen,On Job Training,Morning,5.0


##Modeling

Alasan menggunakan KMeans karna algoritma ini yang paling sederhana dan mudah dipahami oleh saya, tapi jumlah cluster pada soal ini belum diketahui sehingga harus menggunakan metode KElbow untuk menemukannya.

In [ ]:
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X)
df

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/google/colab/_dataframe_summarizer.py:88: UserWarning: Parsing dates in %d/%m/%Y %H:%M format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  cast_date_col = pd.to_datetime(column, errors="coerce")


,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,Customer_City,Product_category,Item_price,connected_handling_time,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score
0,7e9ae164-6a8b-4521-a2d4-58f7c9fff13f,Outcall,Product Queries,Life Insurance,NaN,c27c9bb4-fa36-4140-9f1f-21009254ffdb,NaN,1/8/2023 11:13,1/8/2023 11:47,1-Aug-23,NaN,NaN,NaN,NaN,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,Morning,5.0
1,b07ec1b0-f376-43b6-86df-ec03da3b2e16,Outcall,Product Queries,Product Specific Information,NaN,d406b0c7-ce17-4654-b9de-f08d421254bd,NaN,1/8/2023 12:52,1/8/2023 12:54,1-Aug-23,NaN,NaN,NaN,NaN,Vicki Collins,Dylan Kim,Michael Lee,>90,Morning,5.0
2,200814dd-27c7-4149-ba2b-bd3af3092880,Inbound,Order Related,Installation/demo,NaN,c273368d-b961-44cb-beaf-62d6fd6c00d5,NaN,1/8/2023 20:16,1/8/2023 20:38,1-Aug-23,NaN,NaN,NaN,NaN,Duane Norman,Jackson Park,William Kim,On Job Training,Evening,5.0
3,eb0d3e53-c1ca-42d3-8486-e42c8d622135,Inbound,Returns,Reverse Pickup Enquiry,NaN,5aed0059-55a4-4ec6-bb54-97942092020a,NaN,1/8/2023 20:56,1/8/2023 21:16,1-Aug-23,NaN,NaN,NaN,NaN,Patrick Flores,Olivia Wang,John Smith,>90,Evening,5.0
4,ba903143-1e54-406c-b969-46c52f92e5df,Inbound,Cancellation,Not Needed,NaN,e8bed5a9-6933-4aff-9dc6-ccefd7dcde59,NaN,1/8/2023 10:30,1/8/2023 10:32,1-Aug-23,NaN,NaN,NaN,NaN,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,Morning,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10522,3e5424f6-1134-4228-8a0a-dfcce16e7477,Inbound,Returns,Reverse Pickup Enquiry,Good,9e97d6a3-4c68-4654-903f-50cba17cf7e6,NaN,5/8/2023 22:57,5/8/2023 23:00,5-Aug-23,NaN,NaN,NaN,NaN,Carolyn Reese,Madison Kim,Michael Lee,31-60,Evening,5.0
10523,a006368a-46f2-4260-84fd-b48a7e6f8730,Inbound,Product Queries,Product Specific Information,Very nice,d07fdfdb-bae7-492a-804b-fa546f37ea1a,NaN,6/8/2023 15:59,6/8/2023 16:11,6-Aug-23,NaN,NaN,NaN,NaN,Colleen Savage,Ava Wong,William Kim,On Job Training,Evening,5.0
10524,26f2d90b-4e70-4228-a30c-4d23d2597b8e,Inbound,Returns,Reverse Pickup Enquiry,NaN,808bdcc8-e64d-4808-9dc4-7024726dac9b,NaN,5/8/2023 7:38,5/8/2023 7:38,5-Aug-23,NaN,NaN,NaN,NaN,Rachel Peters,Aiden Patel,John Smith,>90,Morning,5.0
10525,4388f9b2-58fa-43f9-bf1d-ef91c85b4261,Inbound,Product Queries,Product Specific Information,NaN,9d6addee-d279-45da-8cc9-bc258f540048,NaN,5/8/2023 13:53,5/8/2023 14:00,5-Aug-23,NaN,NaN,NaN,NaN,Mark Garcia,Wyatt Kim,Jennifer Nguyen,On Job Training,Morning,5.0


##Evaluasi

Alasan menggunakan evaluasi silhouette score adalah karna metode ini memberikan ukuran tentang seberapa baik data yang dicluster.

In [ ]:
silhouette_avg = silhouette_score(X, clusters)
print("Silhouette Score : ", silhouette_avg)

Silhouette Score :  0.9957970217571049
